In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain_core.language_models import LLM
from typing import List, Optional

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True
)

class Phi2LLM(LLM):
    def __init__(self, model_name: str):
        super().__init__()
        self._tokenizer = AutoTokenizer.from_pretrained(model_name)
        self._model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=bnb_config,   
            device_map="auto", 
            torch_dtype=torch.float16  
        )

    @property
    def _llm_type(self) -> str:
        return "phi2_llm"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        inputs = self._tokenizer(prompt, return_tensors="pt").to(self._model.device)
        with torch.no_grad():
            output = self._model.generate(
                **inputs,
                max_new_tokens=256,
                do_sample=True,
                temperature=0.7,
                pad_token_id=self._tokenizer.eos_token_id
            )
        decoded = self._tokenizer.decode(output[0], skip_special_tokens=True)
        return decoded[len(prompt):]


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Programming\Github\EngenheiroIA-contra-ataca\venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Programming\Github\EngenheiroIA-contra-ataca\venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Programming\Github\EngenheiroIA-contra-ataca\venv\Lib\site-packages\ipykernel\kernelapp.py", line 7

In [2]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = Phi2LLM(model_name="microsoft/phi-2")

fitness_prompt = PromptTemplate(
    input_variables=["question", "user_profile"],
    template="""
You are a friendly fitness assistant. Use the user profile to give precise answers.

User Profile:
{user_profile}

Question:
{question}

Answer in Portuguese, with practical tips on exercises, nutrition or workout planning.
"""
)

chain = LLMChain(llm=llm, prompt=fitness_prompt)


Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.22s/it]
C:\Users\habee\AppData\Local\Temp\ipykernel_29760\513431656.py:21: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=fitness_prompt)


In [3]:
profile = "Idade: 30 anos; Nível: iniciante; Objetivo: perder gordura abdominal."
q1 = "Quais exercícios são mais eficazes para queimar gordura na barriga?"

print(chain.run({"question": q1, "user_profile": profile}))


C:\Users\habee\AppData\Local\Temp\ipykernel_29760\2709792694.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(chain.run({"question": q1, "user_profile": profile}))
c:\Programming\Github\EngenheiroIA-contra-ataca\venv\Lib\site-packages\transformers\integrations\sdpa_attention.py:54: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


## INPUT

##OUTPUT
Nosso assistente de fitness afirma que é importante desenvolver uma rotina de exercícios e alimentos práticos para atender ao objetivo de perder gordura na barriga. Para isso, podemos escolher exercícios com eficácia para barrigar como:

1. Burras: Burras e técnicas de core ou mantejo da core é um melhor forma de perder gordura na barriga. Podemos trabalhar com eles por 10-15 minutos, com 5-10 burras repetitivos e 1-2 tempo limitado.
2. Press: A press é um exercício que pode perder gordura na barriga e fortalece os músculos da pé. Podemos trabalhar com eles por 10-15 minutos, com 10-12 repetitivos e 1-2 tempo limitado.
3. Lunges: Lunges é um exercício que
